In [ ]:
#We mount our google drive to have access to the data files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
import os
import cv2
import random

In [ ]:
#The images obtained from the code have been zipped and uploaded to google drive.
!unzip /content/gdrive/MyDrive/Internetworking/images.zip -d /content/gdrive/MyDrive/Internetworking

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_58/angle_58_122.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_-58/angle_-58_122.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_58/angle_58_123.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_-58/angle_-58_123.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_58/angle_58_124.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_-58/angle_-58_124.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_58/angle_58_125.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_-58/angle_-58_125.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_58/angle_58_126.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images/angle_-58/angle_-58_126.png  
  inflating: /content/gdrive/MyDrive/Internetworking/images

In [ ]:
dataset_path = "/content/gdrive/MyDrive/Internetworking/images"

In [ ]:
def load_data(dataset_path):
    images = []
    labels = []
    for angle_folder in os.listdir(dataset_path):
        angle_path = os.path.join(dataset_path, angle_folder)
        if not os.path.isdir(angle_path):
            continue  # Skip if it's not a directory
        angle_label = int(angle_folder.split('_')[1])  # Extract angle label from folder name
        for image_name in os.listdir(angle_path):
            image_path = os.path.join(angle_path, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (100, 100))  # Resize the image as needed
            images.append(image)
            labels.append(angle_label)
    return np.array(images), np.array(labels)

In [ ]:
# Load the data
images, labels = load_data(dataset_path)

In [ ]:
# Normalize pixel values to be between 0 and 1
images = images.astype('float32') / 255.0


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Change loss function accordingly
              metrics=['mae'])  # You can add more metrics as needed

In [ ]:
# Train the model on all available data
model.fit(images, labels, epochs=100, batch_size=32)

Epoch 1/100
670/670 [==============================] - 5s 5ms/step - loss: 76.1341 - mae: 5.3772
Epoch 2/100
670/670 [==============================] - 4s 5ms/step - loss: 9.4119 - mae: 2.3449
Epoch 3/100
670/670 [==============================] - 4s 5ms/step - loss: 6.2594 - mae: 1.9470
Epoch 4/100
670/670 [==============================] - 3s 5ms/step - loss: 4.9279 - mae: 1.7101
Epoch 5/100
670/670 [==============================] - 3s 5ms/step - loss: 3.9694 - mae: 1.5085
Epoch 6/100
670/670 [==============================] - 4s 5ms/step - loss: 3.5691 - mae: 1.4238
Epoch 7/100
670/670 [==============================] - 3s 5ms/step - loss: 3.0478 - mae: 1.2917
Epoch 8/100
670/670 [==============================] - 3s 5ms/step - loss: 2.5627 - mae: 1.1690
Epoch 9/100
670/670 [==============================] - 3s 5ms/step - loss: 2.1998 - mae: 1.0805
Epoch 10/100
670/670 [==============================] - 4s 5ms/step - loss: 2.0710 - mae: 1.0553
Epoch 11/100
670/670 [================

In [ ]:

model.save("angle_classifier_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
predicted_angle = model.predict(np.expand_dims(cv2.resize(cv2.imread("/content/gdrive/MyDrive/Internetworking/images/angle_20/angle_20_111.png"), (100, 100)) / 255.0, axis=0))[0][0]
print(f"Predicted Angle for is: {predicted_angle}")

1/1 [==============================] - 0s 84ms/step
Predicted Angle for is: 19.52203369140625


In [ ]:
# Path to the directory containing images
images_dir = "/content/gdrive/MyDrive/Internetworking/images"

# List to store actual and predicted angles
actual_angles = []
predicted_angles = []

# Randomly select 10 images from the entire dataset
all_image_paths = []
for angle_folder in os.listdir(images_dir):
    angle_path = os.path.join(images_dir, angle_folder)
    if os.path.isdir(angle_path):
        image_files = [f for f in os.listdir(angle_path) if os.path.isfile(os.path.join(angle_path, f))]
        all_image_paths.extend([os.path.join(angle_path, f) for f in image_files])

selected_image_paths = random.sample(all_image_paths, min(10, len(all_image_paths)))

# Predict angle for each selected image
for image_path in selected_image_paths:
    # Extract actual angle from the directory name
    actual_angle = int(os.path.basename(os.path.dirname(image_path)).split("_")[1])
    # Read and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (100, 100)) / 255.0
    image = np.expand_dims(image, axis=0)
    # Predict angle using the model
    predicted_angle = model.predict(image)[0][0]
    # Append actual and predicted angles to the lists
    actual_angles.append(actual_angle)
    predicted_angles.append(predicted_angle)

# Print actual and predicted angles
for actual_angle, predicted_angle in zip(actual_angles, predicted_angles):
    print(f"Actual Angle: {actual_angle}, Predicted Angle: {predicted_angle}")

1/1 [==============================] - 0s 20ms/step
Actual Angle: -26, Predicted Angle: -26.13861656188965
Actual Angle: 17, Predicted Angle: 16.719186782836914
Actual Angle: -24, Predicted Angle: -23.8083553314209
Actual Angle: -16, Predicted Angle: -16.021455764770508
Actual Angle: -23, Predicted Angle: -22.955167770385742
Actual Angle: -32, Predicted Angle: -31.936359405517578
Actual Angle: 13, Predicted Angle: 13.114638328552246
Actual Angle: 24, Predicted Angle: 23.845073699951172
Actual Angle: -20, Predicted Angle: -19.915428161621094
Actual Angle: 13, Predicted Angle: 13.026424407958984
